# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションに参加するためのエージェントの実装を目的としています。特に、「20の質問」ゲームにおいて、エージェントがキーワードを推測するための戦略を構築しています。

### 問題の概要
このノートブックは、エージェントを利用して、ターゲットとなる単語を20問以内に推測する問題に取り組んでいます。主要なロジックは、二分探索を使用して情報を効率的に取得することにあります。具体的には、AgentAlphaというエージェントが、与えられたキーワードリストの中から正しいキーワードを推測するための質問を生成します。

### 手法とライブラリ
1. **エージェントの設計**: 
   - **AgentAlpha**: アルファベット順の比較と二分探索を用いて、最適な質問を生成します。このエージェントは、ゲームのルールに基づき、ターゲットワードの位置を特定するための質問を行います。プレイヤーは、「はい」または「いいえ」で応答し、それに基づいてさらに質問を続けます。
   - **agent_beta**: シンプルな応答を生成するプレースホルダとして設計されており、条件に基づいて異なる応答を返します。

2. **ライブラリの使用**:
   - **random**: ランダムな選択肢を生成するために使用されています。
   - **re**: 正規表現を用いて質問のフォーマットを解析するために使用されています。
   - **kaggle_environments**: Kaggleの競技用環境を利用し、エージェントをテストするために必要なライブラリです。

3. **エージェントの実行**: 最後に、テスト用の環境を設定し、エージェントを実行してそのパフォーマンスを確認し、結果を描画します。この過程ではダミーエージェント（agent_dummy）が利用され、独自のエージェント（agent_fn）と対戦させています。

このノートブックは、効果的な質問生成と回答解析に焦点を当て、限られた時間内での推測を目的とした戦略的なアプローチを示しています。エージェントは、最適な質問をすることでターゲットの特定を迅速に行えるよう設計されていますが、特定のキーワードリストに依存している点が留意すべき課題です。

---


# 用語概説 
以下は、Jupyter Notebookの内容に関連する機械学習・深層学習の専門用語の簡単な解説です。特に初心者が理解する際につまずきそうな点に焦点を当てています。

1. **エージェント (Agent)**:
   - ゲームや環境内での行動を決定し、他のエージェントや環境と相互作用するプログラム。アルファ・ベータといった異なるタイプのエージェントが存在し、特定の戦略や役割を持つ。

2. **ハンドシェイクプロトコル (Handshake Protocol)**:
   - エージェントが通信を開始する際に行う一連の確認作業。ここでは、エージェントがお互いに自己確認を行い、通信を確立するために使用されます。

3. **正規表現 (Regex)**:
   - 特定の文字列パターンを検索するための強力なツール。ここでは、質問の形式が正しいかどうかを確認するために使用されています。

4. **キーワード (Keyword)**:
   - ゲーム内で推測するターゲットとなる単語やフレーズのこと。エージェントはこのキーワードを使って質問を生成し、相手の回答に基づいて次の行動を決定します。

5. **二分探索 (Binary Search)**:
   - ソートされたリスト内で特定の値を効率的に探す手法。ここでは、キーワードリストの中から適切なキーワードを見つけるために使用されます。

6. **パイプライン (Pipeline)**:
   - 一連の処理やタスクを直列に実行するためのフレームワーク。ここでは、質問に対する回答の処理フローを指します。

7. **推測 (Guess)**:
   - エージェントがターゲットワードを特定するために行う行動。ゲーム内では、質問に対して「はい」または「いいえ」で答えることを介して推測が行われます。

8. **テストワード (Test Word)**:
   - エージェントが現在の質問の答えを基にして推測する具体的な単語。次にどの単語を検討するかを決定するために使用されます。

9. **互換性 (Compatibility)**:
   - システムが異なるバージョンのコンポーネントやエージェント同士で正常に動作する能力を示す。ここでは、異なるエージェント間でスムーズにやり取りできることを指します。

10. **エピソード (Episode)**:
    - 環境での一連の動作や実行を指します。一つのゲームセッションや試行を表し、通常は特定の初期条件からスタートする。

これらの用語は、特定のアルゴリズムや戦略に関連しており、初心者が全体の流れやゲームの構造を理解する上で役立ちます。

---


In [ ]:
# 提出物を作成する際に適用します
# %mkdir /kaggle/working/submission  # 新しいディレクトリを作成します。このディレクトリは提出物用です。

In [ ]:
# %%writefile submission/main.py  # submissionディレクトリ内にmain.pyというファイルを書き込みます。

# Agent Alphaは、アルファベットの順序や比較、二分探索を利用します。
# 最適な検索を行い、20ターン以内に答えを見つけることが保証されています（上限はlog2(n_keywords)+1）。
# 条件としては以下の通りです：
#   * 解決するキーワードは、全ての単語のリストにあり（下記参照）、2**19項目を超えないこと。
#   * 他のチームのプレイヤーもAlphaを使用し、アルファの質問に正しく応答できること。
#
# 解決キーワードがリストになければ、α探索ではキーワードを推測することはできませんが、他の手法と組み合わせることで結果が有用となることもあります。

# ハンドシェイクプロトコルなしに受回答者として受け身でプレイすることも可能です。とはいえ、質問者がハンドシェイクなしにAlphaを試みる可能性は低いでしょう。いずれにせよ、答えるパイプラインの前にregexマッチャーを配置するだけで済みます。実装方法を参照してください。

# それでは、よろしくお願いします。loh-maa


import random
import re

VERSION = 9  # エージェントのバージョンを定義します

# ゲームで期待される全てのキーワードのリスト。エージェントを評価フェーズに提出する前に更新する必要があります。
# ご自身の判断を使用してください。リストが長いほどカバレッジが良くなりますが、収束は遅くなります。カバレッジは収束よりも重要と言えるため、欠ける単語があるよりも、余分な単語がある方が良いです。
allwords = ['xxx', 'remember', 'roll out your own!', 'xxx', 'this list is just for testing', 'advertisement',
            'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx',
            'agave', 'air compressor', ... # 短縮表示のため省略

def agent_beta(obs, cfg):
    """ このエージェント関数はプレースホルダです。自分自身で展開してください！LLMやお好きな代替手段を使用してください。 """

    if obs.turnType == 'ask':
        # LLMや他の方法を使用する
        if len(obs.questions) % 2:
            response = "Alphaは失敗しました、良いことにまだ一手残っています..."
        else:
            response = "私はあなたの個人AIアシスタント、コ-pilotです！次の質問は何ですか？"

    elif obs.turnType == 'answer':
        response = 'yes'

    elif obs.turnType == 'guess':
        if len(obs.questions) % 2:
            # 可愛らしいマーケティングで応じるかもしれません
            responses = ['no alpha, no fun', 'alphaをチェックしてください、向こうの方', 'alphaは命です',
                         'アルファベットの二分探索が助けてくれます', f'クリーミーでミルキーな、アルファ {VERSION}',
                         'カグルをもっと素晴らしいものに']

            response = random.choice(responses)

        else:
            # LLMや他の方法を使用する
            response = "あなたのコ-pilotは近づいていると思います！ **eiffel towel**"

    else:
        assert False

    return response

def answered_yes(obs, question):
    """ 質問がされ、回答が'yes'だったかをチェックします。 """
    try:
        ix = obs.questions.index(question)
        return obs.answers[ix] == 'yes'
    except (ValueError, IndexError):
        return False

class AgentAlpha:

    # 互換性のために固定しています
    HANDSHAKE = 'Is it Agent Alpha?'

    # これは私たちの探索空間で、修正されるので、
    # 一連のゲームを連続でテストするときには再初期化が必要です
    keywords = sorted(allwords)  # キーワードをソートします

    @staticmethod
    def parse_alpha_question(question):
        # アルファ質問に応じるための正規表現を定義します。前のバージョンからの4つのバリエーションを含みます：
        match = re.search(r'keyword.*(?:come before|precede) \"([^\"]+)\" .+ order\?$', question)
        if match:
            # アルファ質問が一致しました
            testword = match.group(1)
            return testword
        else:
            return None

    @staticmethod
    def play(obs, cfg):

        if obs.turnType == 'ask':
            if len(obs.questions) == 0:
                # これは最初の質問になります。
                return AgentAlpha.HANDSHAKE

            elif answered_yes(obs, AgentAlpha.HANDSHAKE) and AgentAlpha.keywords:
                # 1つの固定質問のみを使用します。質問を回転させることに意味はなくなりました。
                testword = AgentAlpha.keywords[len(AgentAlpha.keywords) // 2]
                # 質問フォーマットを固定し、テストワードのみを代入します
                response = f'キーワード（小文字で）が"{testword}"のアルファベット順の前に来ますか？'

            else:
                # キーワードが尽きた場合、ソリューションが onboard に存在しなかったか、
                # 受回答者がミスをした場合、Noneを返しますので、別のアプローチを試すことができます
                response = None

        elif obs.turnType == 'answer':
            # ハンドシェイクの質問が見られた場合、Yesと答え、アルファ質問者が進められるようにします。
            if AgentAlpha.HANDSHAKE == obs.questions[-1]:
                response = 'yes'

            else:
                testword = AgentAlpha.parse_alpha_question(obs.questions[-1])
                if testword is not None:
                    response = 'yes' if obs.keyword.lower() < testword else 'no'
                else:
                    # アルファ質問ではないので、別のアプローチを進めます
                    response = None

        elif obs.turnType == 'guess':
            # 最後の質問がハンドシェイクだった場合、特別な推測を行います
            if obs.questions[-1] == AgentAlpha.HANDSHAKE:
                # それはハンドシェイクラウンドです...
                if obs.answers[-1] == 'yes':
                    response = f"試しましょう.. バージョン {VERSION}"
                else:
                    response = f'eye'

            else:
                # 最後の質問と回答に基づいて空間を二分します
                testword = AgentAlpha.parse_alpha_question(obs.questions[-1])
                if testword:
                    if obs.answers[-1] == 'yes':
                        AgentAlpha.keywords = [k for k in AgentAlpha.keywords if k < testword]
                    else:
                        AgentAlpha.keywords = [k for k in AgentAlpha.keywords if not k < testword]

                if AgentAlpha.keywords:
                    # 前からポップしますが、どちらの側からポップしてもあまり関係ありません
                    response = AgentAlpha.keywords.pop(0)

                else:
                    # キーワードが尽きた場合、別のものを使用します
                    response = None
        else:
            assert False, f'予期しないturnTypeです: {obs.turnType}'

        return response

def agent_fn(obs, cfg):
    """ メインフックです。常に'agent_fn'という名前であり、提出物の最後の関数である必要があります。 """

    try:
        # 常にエージェントAlphaを最初に試みます。意味のあるプレイを行うか、
        # プレイできなかった場合はNoneを返します
        response = AgentAlpha.play(obs, cfg)

        if response is None:
            # それなら他の方法を試すことができます
            response = agent_beta(obs, cfg)

    except Exception:
        import traceback
        traceback.print_exc()
        response = 'no'

    return response

In [ ]:
%%time  # コードの実行時間を計測します。

# テストのみのため、提出時には無効にします
# （「提出ファイルが見つかりません」というエラーが出る場合あり）

import kaggle_environments  # Kaggle環境ライブラリをインポートします。

def agent_dummy(obs, cfg):
    if obs.turnType == "ask":
        response = "Is it a duck?"  # 質問の際に「それはアヒルですか？」と答えます。
    elif obs.turnType == "guess":
        response = "duck"  # 推測の際には「アヒル」と答えます。
    elif obs.turnType == "answer":
        response = "no"  # 回答の際には「いいえ」と答えます。
    else:
        assert False  # 予期しないタイプの場合はエラーを出します。
    return response

# デバッグ用の設定を定義します。
debug_config = {'episodeSteps': 61,  # 初期ステップとラウンドごとに3ステップ（質問/回答/推測）。
                'actTimeout': 60,  # エージェントがラウンドごとに使用できる時間（秒）。デフォルトは60。
                'runTimeout': 1200,  # エピソードにかけられる最大時間（秒）。デフォルトは1200。
                'agentTimeout': 3600}  # 廃止されたフィールド。デフォルトは3600。

env = kaggle_environments.make(environment="llm_20_questions", configuration=debug_config, debug=True)  # 環境を設定します。

# 見えない単語をシミュレーションします。
AgentAlpha.keywords = sorted(random.sample(allwords, 500))  # 全単語から500個をランダムに選んでソートします。

game_output = env.run(agents=[agent_dummy, agent_dummy, agent_fn, agent_fn])  # エージェントを実行します。

# 環境の結果を描画します。
env.render(mode="ipython", width=1080, height=700)  # IPythonモードで環境を描画します。
# out = env.render(mode="ansi")
# print(out)  # ANSIモードで描画した場合の出力を表示します。

In [ ]:
# 提出物を作成する際に有効にします.. pigzの使用は必須ですか？
# !apt install pigz pv  # pigzおよびpvをインストールします。
# !tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission .  # 提出物をtar.gz形式で圧縮します。

---

# コメント 

> ## OminousDude
> 
> あなたやあなたのコードに対して憎悪を抱くわけではありませんが、このコードを使用することはお勧めしません。なぜなら、プライベートリーダーボードではキーワードリスト、キーワードの種類、その他すべてが変わるため、コードは同じようには動作しないからです。再度言いますが、憎悪はありません。良いコードとアイデアには賛成票を入れました！
> 
> 
> 

---